In [ ]:
import networkx as nx
# import matplotlib.pyplot as plt
import pygraphviz as pgv
import panel as pn
from IPython.display import display

from src.catalog_utils import load_catalog

pn.extension()

## Load catalog

In [ ]:
CATALOG = load_catalog()
# CATALOG

In [ ]:
CATALOG.get_product("SW_MAGx_LR_1B").link_files_http

In [ ]:
products = list(CATALOG.products.keys())

## Define subgroups to plot separately

In [ ]:
group_1 = [
    'SW_CFW_SHA_2Y',
    'SW_MCO_SHA_2Y',
    'SW_MLI_SHA_2D',
    'SW_MAGx_LR_1B',
    'SW_MMA_SHA_2C',
    'SW_MTI_SHA_2C',
    'SW_MCO_SHA_2C',
    'SW_MCO_SHA_2X',
    'SW_MLI_SHA_2C',
    'SW_MIO_SHA_2C',
    'SW_MIO_SHA_2E',
    'SW_MCO_SHA_2D',
    'SW_MCO_SHA_2F',
    'SW_MCR_1DM_2_',
    'SW_MIN_1DM_2_',
    'SW_VOBS_1M_2_',
    'SW_VOBS_4M_2_',
    'SW_MMA_SHA_2F',
    'SW_MIO_SHA_2D',
    'SW_MIO_SHA_2E',
    'SW_MLI_SHA_2E',
    'SW_IBIxTMS_2F',
    'SW_EFIx_LP_1B',
]

group_2 = [
    'SW_FAC_TMS_2F',
    'SW_FACxTMS_2F',
    'SW_EFIx_LP_1B',
    'SW_IBIxTMS_2F',
    'SW_IBP_CLI_2_',
    'SW_MAGx_LR_1B',
    'SW_AEJxLPL_2F',
    'SW_AEJxPBL_2F',
    'SW_EEFxTMS_2F',
    'SW_AEJxLPS_2F',
    'SW_AEJxPBS_2F',
    'SW_MCO_SHA_2X',
    'SW_AOBxFAC_2F',
    'SW_IPDxIRR_2F',
    'SW_MITx_LP_2F',
    'SW_PPIxFAC_2F',
    'SW_MITxTEC_2F',
    'SW_TECxTMS_2F',
    'SW_AUXxORBCNT',
    'SW_MODx_SC_1B',
    'SW_GPSx_RO_1B',
]

group_3 = [
    'SW_MODx_SC_1B',
    'SW_GPSx_RO_1B',
    'SW_STRxATT_1B',
    'SW_SC_xDYN_1B',
    'SW_EGF_SHA_2_',
    'SW_SP3xKIN_2_',
    'SW_SP3xCOM_2_',
    'SW_ACCxPOD_2_',
    'SW_ACCxCAL_2_',
    'SW_DNSxPOD_2_',
    'SW_ACCx_AE_2_',
    'SW_DNSxACC_2_',
]

group_4 = [
    'CH_DNS_ACC_2_',
    'CH_TEC_TMS_2F',
    'CH_WND_ACC_2_',
    'CS_MAG',
    'GF_DNSxACC_2_',
    'GF_NE__KBR_2F',
    'GF_TECxTMS_2F',
    'GFx_FGM_ACAL_CORR',
    'GO_MAG_ACAL_CORR',
    'GRACE_x_MAG',
    'GR_DNSxACC_2_',
    'GR_NE__KBR_2F',
    'GR_TECxTMS_2F',
    'GR_WNDxACC_2_',
    'MM_CON_EPH_2_',
]

group_remaining = [_p for _p in CATALOG.product_ids if _p not in group_1 + group_2 + group_3 + group_4]

In [ ]:
product_subset = pn.widgets.MultiChoice(name="input_products", options=CATALOG.product_ids, width=1000, value=group_4)
pn.Column(product_subset, min_height=400)

In [ ]:
product_subset.value

## Define dependencies and categories

In [ ]:
product_dependencies = {
    product: CATALOG.get_product(product).input_products
    for product in CATALOG.products.keys()
}
product_dependencies

In [ ]:
avail_cats ={
    "Advanced": "red",
    "Level1b": "black",
    "Level2daily": "green",
    "Level2longterm": "blue",
    "Multimission": "purple",
}
product_categories = {}
for product in CATALOG.products.keys():
    data_url = CATALOG.get_product(product).link_files_http
    for catname in avail_cats.keys():
        if catname in data_url:
            product_categories[product] = catname
            continue
product_categories

In [ ]:
avail_fast = (
    # "SW_ACCx_PR_1B",
    "SW_ASMxAUX_1B",
    "SW_EFIx_LP_1B",
    "SW_EFIxLPI_1B",
    "SW_GPSxNAV_1B",
    "SW_GPSx_RN_1B",
    "SW_GPSx_RO_1B",
    "SW_LP_x_CA_1B",
    "SW_MAGx_CA_1B",
    "SW_MAGx_LR_1B",
    "SW_MAGx_HR_1B",
    "SW_MODx_SC_1B",
    "SW_SC_xDYN_1B",
    "SW_STRxATT_1B",
    "SW_VFMxAUX_1B",
    "SW_AUXxORBCNT",
)

avail_vires_no_link = (
    "SW_MLI_SHA_2E",
    "SW_IBP_CLI_2_",
    "SW_MODx_SC_1B",
    "SW_AUXxORBCNT",
    "SW_PPIxFAC_2F",
)

In [ ]:
def make_graph(product_dependencies=product_dependencies, product_categories=product_categories, group=group_1, prog="dot"):

    # Select which group to use
    product_dependencies = {_p: product_dependencies[_p] for _p in product_dependencies.keys() if _p in group}
    # product_categories = {_p: product_categories[_p] for _p in product_categories.keys() if _p in group}

    # Build the graph
    G = pgv.AGraph(directed=True)

    # Add all nodes at once
    for node in product_dependencies.keys():
        cat = product_categories[node]
        color = avail_cats.get(cat, "grey")
        shape = "doubleoctagon" if node in avail_fast else "box"
        url = f"https://swarmhandbook.earth.esa.int/catalogue/{node}"
        link_notebook = CATALOG.get_product(node).link_notebook
        link_vires_gui = CATALOG.get_product(node).link_vires_gui
        in_vires_otherwise = node in avail_vires_no_link
        bgcolor = "powderblue" if link_notebook or link_vires_gui or in_vires_otherwise else "white"
        G.add_node(
            node, shape=shape, color=color, URL=url, fillcolor=bgcolor,
            style="filled", fontsize=15, penwidth=3
        )

    # # Add each category of products as a cluster?
    # def _filter_nodes(cat):
    #     return [p for p, c in product_categories.items() if c==cat]
    # for cat, color in avail_cats.items():
    #     G.add_nodes_from(
    #         _filter_nodes(cat), color=color,# cluster=True
    #     )

    # Add dependency edges
    for node, neighbors in product_dependencies.items():
        for neighbor in neighbors:
            G.add_edge(node, neighbor)

    # Visualisation options
    # G.layout(prog="neato", args='-Gmaxiter=100')
    # G.layout(prog="circo")
    G.layout(prog=prog)

    return G

In [ ]:
make

In [ ]:
for i, (group, prog) in enumerate(zip([group_1, group_2, group_3, group_4, group_remaining], ["dot", "dot", "dot", "circo", "circo"])):
    graph_viz = make_graph(group=group, prog=prog)
    graph_viz.draw(f"network_graph_{i}.svg")
    display(graph_viz)

## Make legend manually

In [ ]:
G = pgv.AGraph(directed=True)
for catname, border_color in avail_cats.items():
    shape = "doubleoctagon" if catname == "Level1b" else "box"
    G.add_node(
        catname, shape=shape, color=border_color, style="filled", fillcolor="white", penwidth=3,
    )
G.add_node(
    "Available in VirES", shape="box", color="white", style="filled", fillcolor="powderblue", penwidth=3,
)
G.layout(prog="dot")
G.draw("key_1.svg")
G

In [ ]:
G = pgv.AGraph(directed=True)
G.add_node(
    "Product B", shape="box", color="black", style="filled", fillcolor="white", penwidth=3,
)
G.add_node(
    "Product A", shape="box", color="black", style="filled", fillcolor="white", penwidth=3,
)
G.add_edge("Product B", "Product A", )
# G.add_node(
#     "A depends on B", shape="box", color="white", style="filled", fillcolor="white", penwidth=3,
# )
G.layout(prog="circo")
G.draw("key_2.svg")
G

## Further experiments

In [ ]:
def make_graph_new(product_dependencies=product_dependencies, product_categories=product_categories):

    # Build graph
    G = nx.DiGraph()
    for node in product_dependencies.keys():
        cat = product_categories[node]
        color = avail_cats.get(cat, "grey")
        G.add_nodes_from(
            [
                (node, {"color": color, "shape": "box", "fontsize": 5}),
            ]
        )
    for node, neighbors in product_dependencies.items():
        for neighbor in neighbors:
            G.add_edge(node, neighbor)

    pos = nx.spring_layout(G, seed=42)  # You can choose different layouts as well
    nx.draw(G, pos, with_labels=True, node_size=1500, node_color="skyblue", font_size=12, font_weight="bold")

    return G

make_graph()
